In [3]:

import dspy
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(Path(r"utils/.env"))

True

In [ ]:
from utils.rec_system import RecSysDataset, RecommendationSystem
from utils.retriever import ColbertRetriever
from utils.metrics import precision_at_k
import json

#Complete list of products to retrieve from
with open(Path(r"./products.json"), 'r') as f:
    product_json = json.loads(f.read())

#Input
query = "2 in pvc pipe"
k= 5

#Model initializations
rm = ColbertRetriever(model_name = "colbert", documents = product_json["products"])
lm = dspy.LM(model = os.getenv("BASE_LLM"), api_key = os.getenv("GROQ_API_KEY"), temperature = 0.7)
dspy.settings.configure(lm = lm, rm = rm)


recsys = RecommendationSystem(k=k)
recommended_products = recsys(product_query = query)

#Dataset
dataset = RecSysDataset("recsys_dataset.json")

Loading default colbert model for language en
Default Model: colbert-ir/colbertv2.0
Loading ColBERTRanker model colbert-ir/colbertv2.0 (this message can be suppressed by setting verbose=0)
No device set
Using device mps
No dtype set
Using dtype torch.float32
Loading model colbert-ir/colbertv2.0, this might take a while...
Linear Dim set to: 128 for downcasting


In [24]:
# evaluator = dspy.Evaluate(
#     devset = dataset.test(0, 6),
#     metric = precision_at_k,
#     # num_threads=4,
#     # display_progress=True,
#     display_table=True,
#     # return_all_scores=True,
#     # return_outputs=True,
#     # provide_traceback=True,
# )
# scores = evaluator(recsys)

In [25]:
trainset = dataset.train(0, 6)
optimizer = dspy.teleprompt.LabeledFewShot(
    k=4
)
# optimizer.compile(
#     student = recsys,
#     trainset = trainset,
# )


In [28]:
trainset = dataset.train(0, 6)
optimizer = dspy.teleprompt.BootstrapFewShot(
    metric=precision_at_k,
    max_labeled_demos=2,
    max_bootstrapped_demos=6
)

In [29]:
# compiled_program = optimizer.compile(
#     student = recsys,
#     trainset = trainset
# )

In [30]:
# compiled_program.save("recsys_bootstrapfeshot.json")

In [31]:
# lm.inspect_history(n=1)

In [32]:
# compiled_program.save(path = "recsys_labelled_fewshot.json", save_program=False)

In [33]:
# opt_program = recsys.load(r"./recsys_labelled_fewshot.json")

In [34]:
bs_fewshot_rs = dspy.BootstrapFewShotWithRandomSearch(
    metric = precision_at_k,
    max_bootstrapped_demos=2,
    max_labeled_demos=3,
    max_rounds=1,
    num_candidate_programs=0,
    num_threads=1,
    max_errors=10,
    stop_at_score=None,
    metric_threshold=None,
)

Going to sample between 1 and 2 traces per predictor.
Will attempt to bootstrap 0 candidate sets.


In [35]:
new_program = bs_fewshot_rs.compile(
    student = recsys,
    trainset = dataset.train(0,6),
    valset = dataset.dev(6,8)
)

  0%|          | 0/2 [00:00<?, ?it/s]

2025/03/25 13:35:23 ERROR dspy.utils.parallelizer: Error processing item Example({'product_query': 'I want a 2 in pvc pipe', 'rec_id': '[7, 9, 2]'}) (input_keys={'product_query'}): module 'dspy' has no attribute 'settings'. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 2 (0.0%):   0%|          | 0/2 [00:00<?, ?it/s]

2025/03/25 13:35:23 ERROR dspy.utils.parallelizer: Error processing item Example({'product_query': 'I want a 1 in pvc conduit', 'rec_id': '[8, 4, 1]'}) (input_keys={'product_query'}): module 'dspy' has no attribute 'settings'. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 2 (0.0%): 100%|██████████| 2/2 [00:00<00:00, 189.30it/s]

...

Average Metric: 0.00 / 2 (0.0%):   0%|          | 0/2 [00:00<?, ?it/s]

2025/03/25 13:35:23 ERROR dspy.utils.parallelizer: Error processing item Example({'product_query': 'I want a 1 in pvc conduit', 'rec_id': '[8, 4, 1]'}) (input_keys={'product_query'}): module 'dspy' has no attribute 'settings'. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 2 (0.0%): 100%|██████████| 2/2 [00:00<00:00, 173.18it/s]

2025/03/25 13:35:23 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 2 (0.0%)



Scores so far: [0.0, 0.0, 0.0]
Best score so far: 0.0
3 candidate programs found.


In [36]:
new_program.candidate_programs

[{'score': 0.0,
  'subscores': [0.0, 0.0],
  'seed': -3,
  'program': recommendation_system.predict = Predict(StringSignature(product_query, retrieved_products -> reasoning, recommended_product_ids
      instructions='You are an AI Assistant that recommends products to users based on their queries.'
      product_query = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Product Query:', 'desc': '${product_query}'})
      retrieved_products = Field(annotation=list[Product] required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Retrieved Products:', 'desc': '${retrieved_products}'})
      reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
      recommended_product_ids = Field(annotation=list[int] required=True json_schema_extra={'desc': 'Give me top 3 recommended product ids. You have to return 